<a href="https://colab.research.google.com/github/girinath18/AI_Headshot_RG/blob/Winter/Nokia2API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Run Before the script cell**

In [1]:
!pip install pyngrok
from pyngrok import ngrok
ngrok.set_auth_token("2grhewRoUtjHE4SVcxmbIPobZ9e_7KdSgCrcQoczPKpuwUtpm")
public_url = ngrok.connect(5000).public_url
!git clone https://github.com/girinath18/roop.git
%cd roop

fatal: destination path 'roop' already exists and is not an empty directory.
/content/roop


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install flask_ngrok
!pip install onnxruntime
!pip install customtkinter
!pip install torch torchvision
!pip install flask
!pip install pyngrok --upgrade
!pip install --upgrade pyngrok
!pip install -r /content/roop/requirements.txt

# **creating your Flask app to serve the JSON data**

In [ ]:
!pip install flask flask-ngrok

In [ ]:
from flask import Flask, jsonify
from flask_ngrok import run_with_ngrok
import json
import time
import requests

app = Flask(__name__)
run_with_ngrok(app)


json_path = '/content/drive/MyDrive/templates/categories.json'
with open(json_path) as f:
    data = json.load(f)

@app.route('/categories', methods=['GET'])
def get_categories():
    return jsonify(data)

if __name__ == '__main__':
    try:
        app.run()
    except Exception as e:
        print(f"Error occurred: {e}")

        time.sleep(5)
        app.run()


# **Latest working code**

In [ ]:
from flask import Flask, request, jsonify, send_from_directory, url_for
import os
import shutil
import logging
from pyngrok import ngrok
import subprocess

app = Flask(__name__)

logging.basicConfig(level=logging.INFO)

TEMPLATES_FOLDER_PATH = "/content/drive/MyDrive/templates/florist"
OUTPUT_BASE_FOLDER = "/content/output_Bot"
PERMANENT_OUTPUT_FOLDER = "/content/output_Bot/RGR(Outputs)"

def face_swap(source_image_folder, output_base_folder):
    try:
        output_folder = os.path.join(output_base_folder, "RGR(Outputs)")
        os.makedirs(output_folder, exist_ok=True)
        os.makedirs(PERMANENT_OUTPUT_FOLDER, exist_ok=True)
        logging.info(f"Output folder created at: {output_folder}")

        source_images = os.listdir(source_image_folder)
        if not source_images:
            logging.error("No source images found.")
            return
        logging.info(f"Found {len(source_images)} source images.")

        template_images = os.listdir(TEMPLATES_FOLDER_PATH)
        if not template_images:
            logging.error("No template images found.")
            return
        logging.info(f"Found {len(template_images)} template images.")

        source_image_name = source_images[0]
        source_image_path = os.path.join(source_image_folder, source_image_name)

        if os.path.isfile(source_image_path):
            logging.info(f"Processing source image: {source_image_name}")

            processed_image_paths = set()

            for target_image_name in template_images:
                target_image_path = os.path.join(TEMPLATES_FOLDER_PATH, target_image_name)

                if os.path.isfile(target_image_path):
                    logging.info(f"Processing target image: {target_image_name}")

                    output_image_path = os.path.join(output_folder, f"{source_image_name}_{target_image_name}")
                    permanent_output_image_path = os.path.join(PERMANENT_OUTPUT_FOLDER, f"{source_image_name}_{target_image_name}")

                    if output_image_path in processed_image_paths:
                        logging.info(f"Skipping already processed image: {output_image_path}")
                        continue

                    command = (
                        f'python run.py -s "{source_image_path}" -t "{target_image_path}" '
                        f'-o "{output_image_path}" --keep-fps --execution-threads 14 --many-faces '
                        f'--execution-provider cuda --frame-processor face_swapper face_enhancer '
                        f'--output-video-quality 35 --temp-frame-format jpg --max-memory 46'
                    )
                    logging.info(f"Running command: {command}")

                    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                    stdout, stderr = process.communicate()
                    if process.returncode != 0:
                        logging.error(f"Failed to process target image: {target_image_name}")
                        logging.error(f"Command output: {stderr.decode('utf-8')}")
                    else:
                        logging.info(f"Successfully processed target image: {target_image_name}")
                        logging.info(f"Command output: {stdout.decode('utf-8')}")
                        shutil.copy(output_image_path, permanent_output_image_path)
                        processed_image_paths.add(output_image_path)

                    if len(processed_image_paths) >= len(template_images):
                        break

    except Exception as e:
        logging.exception(f"An error occurred: {e}")

@app.route('/faceswap', methods=['POST'])
def faceswap_api():
    try:
        source_image_folder = "/content/source_images"
        os.makedirs(source_image_folder, exist_ok=True)

        source_images = request.files.getlist('source_images')
        if not source_images:
            logging.error("No images uploaded.")
            return jsonify({"error": "No images uploaded"}), 400

        for source_image in source_images:
            source_image.save(os.path.join(source_image_folder, source_image.filename))
            logging.info(f"Uploaded image: {source_image.filename}")

        face_swap(source_image_folder, OUTPUT_BASE_FOLDER)

        shutil.rmtree(source_image_folder)

        return jsonify({"message": "Face swap process initiated."}), 200

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

@app.route('/output_images', methods=['GET'])
def output_images_api():
    try:
        if not os.path.exists(PERMANENT_OUTPUT_FOLDER):
            logging.error("Output folder does not exist.")
            return jsonify({"error": "Output folder does not exist"}), 404

        output_images = os.listdir(PERMANENT_OUTPUT_FOLDER)
        if not output_images:
            logging.info("No output images found.")
            return jsonify({"message": "No output images found."}), 200

        image_urls = [
            url_for('get_output_image', filename=img, _external=True)
            for img in output_images
        ]
        return jsonify({"images": image_urls})

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

@app.route('/output_images/<path:filename>', methods=['GET'])
def get_output_image(filename):
    try:
        if os.path.exists(os.path.join(PERMANENT_OUTPUT_FOLDER, filename)):
            return send_from_directory(PERMANENT_OUTPUT_FOLDER, filename)
        else:
            return jsonify({"error": "File not found"}), 404
    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

if __name__ == '__main__':
    url = ngrok.connect(5000)
    print(f" * Ngrok tunnel URL: {url}")

    app.run(port=5000)


 * Ngrok tunnel URL: NgrokTunnel: "https://9845-34-125-14-219.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Failed to process target image: Florist (16).jpg
Downloading: 529MB [00:02, 259MB/s]                           
Downloading: 332MB [00:01, 195MB/s]                           
100%|██████████| 281857/281857 [00:04<00:00, 67461.90KB/s]
Downloading...
From: https://github.com/bhky/opennsfw2/releases/download/v0.1.0/open_nsfw_weights.h5
To: /root/.opennsfw2/weights/open_nsfw_weights.h5
100%|██████████| 24.2M/24.2M [00:00<00:00, 47.4MB/s]
100%|██████████| 104M/104M [00:00<00:00, 235MB/s] 
100%|██████████| 81.4M/81.4M [00:00<00:00, 239MB/s]
Segmentation fault (core dumped)

ERROR:root:Failed to process target image: Florist (17).jpg
ERROR:root:Command output: Segmentation fault (core dumped)

ERROR:root:Failed to process target image: Florist (18).jpg
ERROR:root:Command output: Se

# **Sample**

In [4]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Global variable to store the folder path
folder_path = None

@app.route('/set_folder_path', methods=['GET'])
def set_folder_path():
    global folder_path
    folder_path = request.args.get('path')
    if not folder_path:
        return jsonify({"error": "No path provided"}), 400
    return jsonify({"message": f"Folder path set to: {folder_path}"}), 200

@app.route('/get_folder_path', methods=['GET'])
def get_folder_path():
    if not folder_path:
        return jsonify({"error": "No folder path set"}), 404
    return jsonify({"folder_path": folder_path}), 200

if __name__ == '__main__':
    app.run(port=5000, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
